<h1 style="color:green;"><u>SPOTIFY DATA EXTRACTION FROM SPOTIFY API</u></h1>
    
 <p> In this python notebook we will retrieve a particular playlist data from the
     Spotify API using the spotipy module from Python.
 </p>
 <p> For this project we will will utilize the top 1000+ global songs playlist </p>
    <a href="https://open.spotify.com/playlist/7xNEW3kVH1KXZ3yevHz5u4">Global TOP 1000+ songs</a>
 <p> Before starting the extraction process, install the spotipy module using pip</p>

In [1]:
#Importing the pandas library for dataframe manipulation
#!pip install spotipy
#Import the SpotifyClientCredentials method to set up the credentials and setting up the authorization
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

<p><h3>
    <u>Create a developer account on</u><a> https://developer.spotify.com/dashboard/</a></h3>
    <p>
    <ul>
    <li>Select the Create app option and give a name to the app and click create</li>
    <li>On creation of the app we are provided with the credentials for authorization</li>
    <li>Initialize the client id and client secret id into 2 variables and then we will pass these into the 
        credential manager function as a parameter</li>
    </ul>
    </p>
    </p>

In [2]:
#The client id and client secret id for my app
cid='0e6ab3c6995f49b4a3ae2c74fc8ba8a2'
sec_id='40856ab82c7b441e81aa223f5e6773d4'

In [3]:
#Authorization
client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=sec_id)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [4]:
#This is the playlist id which we will use to get the playlist data into csv
name = "7xNEW3kVH1KXZ3yevHz5u4" 

<p>After authorization we would get the json object of the playlist from the spotify object and 
    we will iterate through the json file to get the desired features of the tracks into the csv
    and this function returns the csv file of the playlist
</p>
<p> Info about few spotipy methods</p>
<p>
    Method- playlist_tracks(playlist_id, fields=None, limit=100, offset=0, market=None)
    <br>
    Description- Get full details of the tracks of a playlist.<br>
    Parameters:
     <ul>
         <li>playlist_id - the id of the playlist</li>
     <li>fields - which fields to return</li>
     <li>limit - the maximum number of tracks to return</li>
     <li>offset - the index of the first track to return</li>
     <li>market - an ISO 3166-1 alpha-2 country code.</li>
     </ul>
 </p>

In [5]:
#CSV Function
def get_playlist_tracks(playlist_id):
    results = sp.playlist_tracks(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    results = tracks    

    playlist_tracks_id = []
    playlist_tracks_titles = []
    playlist_tracks_artists = []
    playlist_tracks_first_artists = []
    playlist_tracks_first_release_date = []
    playlist_tracks_popularity = []

    for i in range(len(results)):
        if i == 0:
            playlist_tracks_id = results[i]['track']['id']
            playlist_tracks_titles = results[i]['track']['name']
            playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
            playlist_tracks_popularity = results[i]['track']['popularity']

            artist_list = []
            for artist in results[i]['track']['artists']:
                artist_list= artist['name']
            playlist_tracks_artists = artist_list

            features = sp.audio_features(playlist_tracks_id)
            features_df = pd.DataFrame(data=features, columns=features[0].keys())
            features_df['title'] = playlist_tracks_titles
            features_df['all_artists'] = playlist_tracks_artists
            features_df['popularity'] = playlist_tracks_popularity
            features_df['release_date'] = playlist_tracks_first_release_date
            #Creating the features dataframe
            features_df = features_df[['id', 'title', 'all_artists', 'popularity', 'release_date',
                                       'danceability', 'energy', 'key', 'loudness',
                                       'mode', 'acousticness', 'instrumentalness',
                                       'liveness', 'valence', 'tempo',
                                       'duration_ms', 'time_signature']]
            continue
        else:
            try:
                playlist_tracks_id = results[i]['track']['id']
                playlist_tracks_titles = results[i]['track']['name']
                playlist_tracks_first_release_date = results[i]['track']['album']['release_date']
                playlist_tracks_popularity = results[i]['track']['popularity']
                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list= artist['name']
                playlist_tracks_artists = artist_list
                features = sp.audio_features(playlist_tracks_id)
                new_row = {'id':[playlist_tracks_id],
               'title':[playlist_tracks_titles],
               'all_artists':[playlist_tracks_artists],
               'popularity':[playlist_tracks_popularity],
               'release_date':[playlist_tracks_first_release_date],
               'danceability':[features[0]['danceability']],
               'energy':[features[0]['energy']],
               'key':[features[0]['key']],
               'loudness':[features[0]['loudness']],
               'mode':[features[0]['mode']],
               'acousticness':[features[0]['acousticness']],
               'instrumentalness':[features[0]['instrumentalness']],
               'liveness':[features[0]['liveness']],
               'valence':[features[0]['valence']],
               'tempo':[features[0]['tempo']],
               'duration_ms':[features[0]['duration_ms']],
               'time_signature':[features[0]['time_signature']]
               }

                dfs = [features_df, pd.DataFrame(new_row)]
                features_df = pd.concat(dfs, ignore_index = True)
            except:
                continue
                #saving the csv as spot_data.csv
        features_df.to_csv("spot_data.csv")
    return features_df

In [7]:
#Calling the function
get_playlist_tracks(name)

,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,78f9pUmqxdrPpUeidwwagK,Levitate - From The Original Motion Picture “P...,Imagine Dragons,0,2016-11-29,0.310,0.681,11,-5.520,1,0.03310,0.000000,0.1850,0.2330,106.784,198173,5
1,3NdDpSvN911VPGivFlV5d0,I Don’t Wanna Live Forever (Fifty Shades Darke...,Taylor Swift,0,2016-12-09,0.735,0.451,0,-8.374,1,0.06310,0.000013,0.3250,0.0862,117.973,245200,4
2,33koOQs551ijjVmLbmrcDc,Yellow - Live from Spotify London,Coldplay,58,2016-12-16,0.223,0.439,11,-10.456,1,0.44700,0.014100,0.9260,0.1190,165.899,252467,4
3,5NQbUaeTEOGdD6hHcre0dZ,679 (feat. Remy Boyz),Remy Boyz,73,2015-06-29,0.618,0.717,7,-5.738,1,0.00256,0.000000,0.6250,0.6030,190.050,196693,4
4,7rklm17yoayxw45lpzv5uv,Se på mig nu (feat. Linnea Henriksson) - Singe...,Linnea Henriksson,52,2016-10-28,0.622,0.729,0,-7.929,0,0.10800,0.000006,0.1630,0.5320,106.053,250040,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1951,059ACLUOyEcdruA2m9f2jd,Unstoppable,Sia,66,2016-02-18,0.468,0.779,9,-4.839,1,0.11200,0.000373,0.1010,0.2600,173.799,217747,4
1952,3BiiSLo0wyHAY7lpCEO6ge,Be Right Now,Ed Sheeran,63,2021-10-29,0.501,0.684,11,-6.706,1,0.00494,0.000478,0.0524,0.4420,131.952,211080,4
1953,0rt63HYAAIzUZo5O2D0uA6,Visiting Hours,Ed Sheeran,67,2021-10-29,0.471,0.396,8,-6.654,1,0.77000,0.000000,0.0729,0.2630,149.609,215507,4
1954,1fEgtkMkRbNN1aaxHTmTMY,Kevlarsjäl,kent,37,1999-12-06,0.511,0.728,1,-5.172,0,0.02330,0.000000,0.1010,0.3510,135.984,266565,4


In [8]:
#Reading the generated CSV
df_spot=pd.read_csv("spot_data.csv")
df_spot.head()

,Unnamed: 0,id,title,all_artists,popularity,release_date,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0,78f9pUmqxdrPpUeidwwagK,Levitate - From The Original Motion Picture “P...,Imagine Dragons,0,2016-11-29,0.310,0.681,11,-5.520,1,0.03310,0.000000,0.185,0.2330,106.784,198173,5
1,1,3NdDpSvN911VPGivFlV5d0,I Don’t Wanna Live Forever (Fifty Shades Darke...,Taylor Swift,0,2016-12-09,0.735,0.451,0,-8.374,1,0.06310,0.000013,0.325,0.0862,117.973,245200,4
2,2,33koOQs551ijjVmLbmrcDc,Yellow - Live from Spotify London,Coldplay,58,2016-12-16,0.223,0.439,11,-10.456,1,0.44700,0.014100,0.926,0.1190,165.899,252467,4
3,3,5NQbUaeTEOGdD6hHcre0dZ,679 (feat. Remy Boyz),Remy Boyz,73,2015-06-29,0.618,0.717,7,-5.738,1,0.00256,0.000000,0.625,0.6030,190.050,196693,4
4,4,7rklm17yoayxw45lpzv5uv,Se på mig nu (feat. Linnea Henriksson) - Singe...,Linnea Henriksson,52,2016-10-28,0.622,0.729,0,-7.929,0,0.10800,0.000006,0.163,0.5320,106.053,250040,4


In [9]:
df_spot.shape

(1956, 18)